**回测（Backtesting）**：基于过去发生的历史信息产生历史交易，并考察这些交易业绩的过程。

# 回测偏差

大多数的偏差会使得策略的表现变好，从而回测结果应当视为策略真实表现的理想上界。偏差无法彻底消除，只能尽量减小。

## 过拟合偏差（Optimisation Bias, Data-Snooping Bias）

该偏差来自于为了使得策略的表现可被接受而不断地调整策略参数的行为。减小过拟合偏差的方法有：尽量降低模型的复杂性和参数的数目，尽量增加训练集的样本点以及敏感性分析（Sensitivity Analysis）。

**尽量降低模型的复杂性和参数的数目**

一般而言，策略的规则越多，模型的参数越多，就越有可能遭遇过拟合偏差，能经得起时间考验的往往是简单的模型。奥卡姆剃刀原理不仅在科学上有效，在金融上也是如此。

相比于线性模型，非线性模型更容易产生过拟合偏差，因为其更复杂，参数往往也更多。策略的盈利不是来自于精巧复杂的模型设置，而是来源于被人们所忽视的市场的内在无效性。

对于机器学习模型的认识（\cite{Chan:2009}）：机器学习模型简单的讲就是一个常使用包含众多参数的函数来拟合历史数据，然而能够获取到的在统计学意义上相互独立的金融数据的数量非常有限，从而约束了机器学习模型的使用。通常有效的机器学习方法包含的特征：
* 基于正确的计量经济学或理论基础，而不是随便发现的模式；
* 需要用历史数据拟合的参数较少；
* 只用到了线性回归，并未使用复杂的非线性函数；
* 概念上简单；
* 所有优化都必须在移动窗口中实现，并且这种优化的结果必须不断被未来未知的数据所证实。

**尽量增加训练集的样本点**

样本量的经验法则（\cite{Chan:2009}）：通常假定优化参数所需的数据点个数是模型中自由参数数量的 252 倍（一年交易日数为 252）。

**敏感性分析**

敏感性分析绘制策略表现相对于参数取值的图像，其基本要求是该图像比较光滑。

参数优化并不一定是挑选那个回测业绩最好的参数集，通常更应该是基于不同参数集的某种平均。或者考虑将资金分配到不同的参数集的策略上，这种资金在参数上的平均化将进一步确保模型的真是业绩与回测业绩不会相差太大。


## 前视偏差（Look-Ahead Bias）

前视偏差指在构建当前模型做出预测时使用了未来的信息。

前视偏差的来源有：程序错误，参数计算使用了全样本等

## 幸存者偏差（Survivorship Bias）

本质上，幸存者偏差是前视偏差的特殊形式。

这种偏差在策略有“价值”偏好时（倾向于买便宜的股票）尤为突出，因为有些股票便宜的原因是公司即将破产。（\cite{Chan:2009}）

## 认知偏差（Cognitive Bias）

# 交易成本（Transaction Cost）

交易成本主要包括：**手续费（Commission）**，**流动性成本**，**延时成本**，**机会成本**等。

## 流动性成本

或称为冲击成本。

**市场冲击（Market Impact）**：自身指令所引起的市场价格变动效应。

## 延时成本

或称为等待成本。

**滑点（Slippage）**

滑点是指触发指令的价格与交易实际执行的价格之差。滑点大小跟资产波动性，交易系统和交易所之间的延迟（Latency）以及策略类型有关。

动量型策略可能会承受更高的滑点，反转型策略则反之。


经验法则（\cite{Chan:2009}）：如果指令规模不比最优买卖报价的平均规模大很多，交易成本通常可以用平均买卖价差的一半加上佣金来估计。

# 数据处理

## 股票除权除息

假设除权除息日为 T，该日每股产生的现金为 d（股息，现金红利），每股调整为 N 股（拆股，并股，送股，转增股），如果为了使得调整前后收益率保持不变，则：
* 前复权：T 日之前的所有股票价格应该乘以$\frac{P_{T-1}-d}{N\cdot P_{T-1}}$；
* 后复权：T 日及其之后的所有股票价格应该乘以$\frac{N\cdot P_{T-1}}{P_{T-1}-d}$。

如果为了使得调整前后价格变化量保持不变，则：
* 前复权：T 日之前的所有股票价格应该减去$P_{T-1} - \frac{P_{T-1}-d}{N}$；
* 后复权：T 日及其之后的所有股票价格应该加上$P_{T-1} - \frac{P_{T-1}-d}{N}$。

这种情况下，价格有可能为负，这时需要对所有的价格加上一个足够大的正数以保证价格为正。


## 期货连续合约

假设在 T 日发生了合约切换，老合约的切换价格为 $P_T$，新合约的切换价格为 $Q_T$，如果为了使得调整前后收益率保持不变，则：
* 前复权：T 日之前的价格应该乘以$\frac{Q_T}{P_T}$；
* 后复权：T 日及其之后的价格应该乘以$\frac{P_T}{Q_T}$。

如果为了使得调整前后价格变化量保持不变，则：
* 前复权：T 日之前的价格应该减去$P_T - Q_T$；
* 后复权：T 日及其之后的价格应该加上$P_T - Q_T$。

这种情况下，价格有可能为负，这时需要对所有的价格加上一个足够大的正数以保证价格为正。

通常使用的是保持收益率不变的调整，但如果策略的逻辑依赖于证券的价格差，则应该选择保持价格变化量不变的方式。


## 结算价

在大多数使用日数据的策略来说，期货价格应该使用结算价，对于相同标的的两个期货合约，结算价可以保证使用的是同时的价格。

# References

[<a id="cit-Chan:2009" href="#call-Chan:2009">Chan:2009</a>] Ernest Chan, ``_Quantitative trading: how to build your own algorithmic trading business_'',  2009.

